In [ ]:
from typing import Sequence, Tuple, Dict, Any, Optional
import os
import dotenv

import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import lovelyplots
import pyemma
import scipy.spatial
plt.style.use('ipynb')

from jamun.metrics._ramachandran import compute_JS_divergence_of_ramachandran, num_dihedrals
from jamun.utils import get_wandb_run_config, convert_to_one_letter_code, convert_to_three_letter_code
from jamun.data import parse_datasets_from_directory

In [ ]:
JAMUN_DATA_PATH = os.environ.get("JAMUN_DATA_PATH", dotenv.get_key("../.env", "JAMUN_DATA_PATH"))
JAMUN_DATA_PATH

Weights and Biases run paths for the sampling runs!

In [24]:
wandb_sample_run_paths = [
    "prescient-design/jamun/4rnu5lxp"
]

In [25]:
def find_project_root() -> str:
    """Returns the path to the root of the project."""
    current_dir = os.path.dirname(os.path.abspath(''))
    while not os.path.exists(os.path.join(current_dir, "pyproject.toml")):
        current_dir = os.path.dirname(current_dir)
    return current_dir

def get_peptides_in_run(wandb_sample_run_path: str) -> Tuple[Sequence[str], str]:
    """Returns the list of peptides sampled in a run and the output directory where they are stored."""

    cfg = get_wandb_run_config(wandb_sample_run_path)
    output_dir = os.path.join(cfg["paths"]["run_path"])
    if output_dir.startswith("."):
        # Path is relative to the project root.
        output_dir = os.path.join(find_project_root(), output_dir)
    output_dir = os.path.abspath(output_dir)

    if not os.path.exists(output_dir):
        raise ValueError(f"Output directory {output_dir} not found.")
    
    peptides_in_run = sorted(os.listdir(os.path.join(output_dir, "sampler")))
    if len(peptides_in_run) == 0:
        raise ValueError(f"No sampled peptides found in {wandb_sample_run_path}")

    return peptides_in_run, output_dir


def get_JAMUN_trajectory_files(wandb_sample_run_paths: Sequence[str]) -> Dict[str, Dict[str, str]]:
    """Returns a dictionary mapping peptide names to the path of the PDB file containing the predicted structure."""

    files = {}
    for wandb_sample_run_path in wandb_sample_run_paths:

        # Get the list of peptides sampled in the run and the output directory where they are stored.
        peptides_in_run, output_dir = get_peptides_in_run(wandb_sample_run_path)

        # Check that there are no duplicates amongst runs.
        for peptide in peptides_in_run:
            if peptide in files:
                raise ValueError(f"Peptide {peptide} found in multiple runs")

            files[peptide] = {
                "dcd": f"{output_dir}/sampler/{peptide}/predicted_samples/dcd/joined.dcd",
                "pdb": f"{output_dir}/pdbs/{peptide}-modified.pdb",
            }

    return files


def get_JAMUN_trajectories(wandb_sample_run_paths: Sequence[str]) -> Dict[str, md.Trajectory]:
    """Returns a dictionary mapping peptide names to the sampled MDTraj trajectory."""
    files = get_JAMUN_trajectory_files(wandb_sample_run_paths)
    trajs = {}
    for peptide, peptide_files in files.items():
        trajs[peptide] = md.load_dcd(peptide_files["dcd"], top=peptide_files["pdb"])
    return trajs

def get_Timewarp_trajectories(data_path: str, peptide_type: str, filter_codes: Optional[Sequence[str]] = None) -> Dict[str, md.Trajectory]:
    """Returns a dictionary mapping peptide names to the Timewarp MDTraj trajectory."""
    if peptide_type == "2AA":
        peptide_type_dir = "2AA-1-large"
    elif peptide_type == "4AA":
        peptide_type_dir = "4AA-large"
    else:
        raise ValueError(f"Invalid peptide type: {peptide_type}")

    one_letter_filter_codes = [''.join([convert_to_one_letter_code(aa) for aa in code]) for code in filter_codes]
    assert len(set(one_letter_filter_codes)) == len(one_letter_filter_codes), "Filter codes must be unique"

    datasets = parse_datasets_from_directory(
        root=f"{data_path}/timewarp/{peptide_type_dir}/test/",
        traj_pattern="^(.*)-traj-arrays.npz",
        pdb_pattern="^(.*)-traj-state0.pdb",
        filter_codes=one_letter_filter_codes,
    )

    # Remap keys.
    one_letter_filter_codes_map = dict(zip(one_letter_filter_codes, filter_codes))
    return {one_letter_filter_codes_map[dataset.label()]: dataset.trajectory for dataset in datasets}


def get_OpenMM_trajectories(data_path: str, filter_codes: Optional[Sequence[str]] = None) -> Dict[str, md.Trajectory]:
    """Returns a dictionary mapping peptide names to the Timewarp MDTraj trajectory."""
    three_letter_filter_codes = ['_'.join([convert_to_three_letter_code(aa) for aa in code]) for code in filter_codes]
    assert len(set(three_letter_filter_codes)) == len(three_letter_filter_codes), "Filter codes must be unique"

    datasets = parse_datasets_from_directory(
        root=f"{data_path}/capped_diamines/timewarp_splits/test",
        traj_pattern="^(.*).xtc",
        pdb_pattern="^(.*).pdb",
        filter_codes=three_letter_filter_codes,
    )

    # Remap keys.
    three_letter_filter_codes_map = dict(zip(three_letter_filter_codes, filter_codes))
    return {three_letter_filter_codes_map[dataset.label()]: dataset.trajectory for dataset in datasets}


def get_TBG_trajectories(root: str) -> Dict[str, md.Trajectory]:
    """Returns a dictionary mapping peptide names to the TBG MDTraj trajectory."""
    true_paths = sorted(os.listdir(root))
    return {peptide: md.load(f"{root}/{peptide}/tbg_samples/pdb/joined.pdb") for peptide in true_paths}

In [26]:
def get_featurized_traj(traj: md.Trajectory, cossin: bool = False):
    """
    Featurize an MDTraj trajectory with backbone and sidechain torsion angles.
    
    Args:
        traj (mdtraj.Trajectory): Input trajectory to featurize
        cossin (bool): Whether to transform angles to cosine/sine pairs
        
    Returns:
        tuple: (feat, featurized_traj) where feat is the PyEMMA featurizer 
        and featurized_traj is the transformed trajectory data
    """
    feat = pyemma.coordinates.featurizer(traj.topology)
    feat.add_backbone_torsions(cossin=cossin)
    feat.add_sidechain_torsions(cossin=cossin)
    featurized_traj = feat.transform(traj)
    return feat, featurized_traj

In [ ]:
JAMUN_trajs = get_JAMUN_trajectories(wandb_sample_run_paths)
peptides = ["AC", "AD"]
JAMUN_trajs = {peptide: JAMUN_trajs[peptide] for peptide in peptides}
JAMUN_trajs

In [ ]:
Timewarp_trajs = get_Timewarp_trajectories(JAMUN_DATA_PATH, peptide_type="2AA", filter_codes=list(JAMUN_trajs.keys()))
Timewarp_trajs

In [ ]:
OpenMM_trajs = get_OpenMM_trajectories(JAMUN_DATA_PATH, filter_codes=list(JAMUN_trajs.keys()))
OpenMM_trajs

In [30]:
# dir = "/data/bucket/kleinhej//capped_diamines/timewarp_splits/test/"
# for filename in os.listdir(dir):
#     peptide = filename.split(".")[0]
#     traj = md.load_xtc(f"{dir}/{peptide}.xtc", top=f"{dir}/{peptide}.pdb")
#     print(peptide, traj.n_frames, "frames")
#     del traj


In [31]:
# TBG_trajs = get_TBG_trajectories("tbg_samples")

In [ ]:
fig, axs = plt.subplots(nrows=len(JAMUN_trajs), ncols=2, figsize=(12, 6), squeeze=False)
for i, peptide in enumerate(JAMUN_trajs):

    Timewarp_feat, Timewarp_featurized_traj = get_featurized_traj(Timewarp_trajs[peptide])
    pyemma.plots.plot_feature_histograms(Timewarp_featurized_traj, feature_labels=Timewarp_feat.describe(), ax=axs[i, 0])

    JAMUN_feat, JAMUN_featurized_traj = get_featurized_traj(JAMUN_trajs[peptide])
    pyemma.plots.plot_feature_histograms(JAMUN_featurized_traj, feature_labels=JAMUN_feat.describe(), ax=axs[i, 1])

    peptide_3letter = "-".join([convert_to_three_letter_code(aa) for aa in peptide])
    axs[i, -1].text(1.1, 0.5, peptide_3letter, rotation=90, verticalalignment='center', horizontalalignment='center', transform=axs[i, -1].transAxes)

axs[0, 0].set_title("Uncapped 2AA Reference")
axs[0, 1].set_title("JAMUN")
plt.tight_layout()

In [ ]:
for i, peptide in enumerate(JAMUN_trajs):
    Timewarp_feat, Timewarp_featurized_traj = get_featurized_traj(Timewarp_trajs[peptide])
    JAMUN_feat, JAMUN_featurized_traj = get_featurized_traj(JAMUN_trajs[peptide])

    print("peptide", peptide)
    for j, feat in enumerate(Timewarp_feat.describe()):
        Timewarp_p = np.histogram(Timewarp_featurized_traj[:,j], range=(-np.pi, np.pi), bins=100)[0]
        JAMUN_p = np.histogram(JAMUN_featurized_traj[:,j], range=(-np.pi, np.pi), bins=100)[0]
        print(feat, "JSD:", scipy.spatial.distance.jensenshannon(Timewarp_p, JAMUN_p))


In [ ]:
fig, axs = plt.subplots(nrows=len(JAMUN_trajs), ncols=2, figsize=(12, 6), squeeze=False)

for i, peptide in enumerate(JAMUN_trajs):
    Timewarp_feat, Timewarp_featurized_traj = get_featurized_traj(Timewarp_trajs[peptide], cossin=True)
    JAMUN_feat, JAMUN_featurized_traj = get_featurized_traj(JAMUN_trajs[peptide], cossin=True)
    
    tica = pyemma.coordinates.tica(Timewarp_featurized_traj, lag=1000, kinetic_map=True)
    Timewarp_tica = tica.transform(Timewarp_featurized_traj)
    JAMUN_tica = tica.transform(JAMUN_featurized_traj)

    pyemma.plots.plot_free_energy(Timewarp_tica[:,0], Timewarp_tica[:,1], cmap="plasma", ax=axs[i, 0])
    axs[i, 0].set_title("Uncapped 2AA Reference")

    pyemma.plots.plot_free_energy(JAMUN_tica[:,0], JAMUN_tica[:,1], cmap="plasma", ax=axs[i, 1])
    axs[i, 1].set_title("JAMUN")

    peptide_3letter = "-".join([convert_to_three_letter_code(aa) for aa in peptide])
    axs[i, -1].text(1.4, 0.5, peptide_3letter, rotation=90, verticalalignment='center', horizontalalignment='center', transform=axs[i, -1].transAxes)

plt.tight_layout()

In [35]:
def plot_ramachandran_contour(traj: md.Trajectory, dihedral_index: int, ax: Optional[plt.Axes] = None) -> plt.Axes:
    """Plots the Ramachandran contour plot of a trajectory."""

    if ax is None:
        _, ax = plt.subplots(figsize=(10,10))
    
    _, phi = md.compute_phi(traj)
    _, psi = md.compute_psi(traj)
    H, xedges, yedges = np.histogram2d(phi.T[dihedral_index], psi.T[dihedral_index], bins=np.linspace(-np.pi,np.pi,50))
    pmf = -np.log(H.T)+np.max(np.log(H.T))

    im = ax.contourf(xedges[:-1], yedges[:-1], pmf, cmap='viridis', levels=50)
    contour = ax.contour(xedges[:-1], yedges[:-1], pmf, colors='white', linestyles='solid', levels=30, linewidths=0.25)

    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel("$\phi$")
    ax.set_ylabel("$\psi$")
    
    tick_eps = 0.1
    ticks = [-np.pi + tick_eps, -np.pi/2, 0, np.pi/2, np.pi - tick_eps]
    tick_labels = ["$-\pi$", "$-\pi/2$", "$0$", "$\pi/2$", "$\pi$"]
    ax.set_xticks(ticks, tick_labels)
    ax.set_yticks(ticks, tick_labels)
    
    return ax

In [ ]:
n_dihedrals = num_dihedrals(next(iter(JAMUN_trajs.values())))
print("n_dihedrals", n_dihedrals)
if n_dihedrals % 2 == 0:
    label_offset = 1.0
else:
    label_offset = 0.5
print("label_offset", label_offset)


fig, axs = plt.subplots(len(JAMUN_trajs), 2 * n_dihedrals, figsize=(16, 8), squeeze=False)
for i, peptide in enumerate(JAMUN_trajs):
    for j in range(n_dihedrals):
        plot_ramachandran_contour(Timewarp_trajs[peptide], j, axs[i, j])
        plot_ramachandran_contour(JAMUN_trajs[peptide], j, axs[i, j + n_dihedrals])

    # Add labels.
    axs[0, n_dihedrals // 2].text(label_offset, 1.1, "Uncapped 2AA Reference", horizontalalignment='center', verticalalignment='center', transform=axs[0, n_dihedrals // 2].transAxes)
    axs[0, n_dihedrals // 2 + n_dihedrals].text(label_offset, 1.1, "JAMUN", horizontalalignment='center', verticalalignment='center', transform=axs[0, n_dihedrals // 2 + n_dihedrals].transAxes)
    
    peptide_3letter = "-".join([convert_to_three_letter_code(aa) for aa in peptide])
    axs[i, -1].text(1.1, 0.5, peptide_3letter, rotation=90, verticalalignment='center', horizontalalignment='center', transform=axs[i, -1].transAxes)

plt.tight_layout()
plt.show()

In [37]:
def compute_JS_divergence_vs_time(traj: md.Trajectory, reference_traj: md.Trajectory, num_steps: int) -> np.ndarray:
    """Computes the JS divergence between the Ramachandran distributions of a trajectory and a reference trajectory at different time points."""
    steps = np.linspace(0, len(traj), (num_steps + 1)).astype(int)[1:]

    js_divs = [
        compute_JS_divergence_of_ramachandran(
            traj[:step],
            reference_traj,
        )
        for step in steps
    ]
    return np.array(js_divs)

def compute_JS_divergences_vs_time(trajs: Dict[str, md.Trajectory], reference_trajs: Dict[str, md.Trajectory], num_steps: int) -> Dict[str, np.ndarray]:
    """Computes the JS divergence between the Ramachandran distributions of a set of trajectories and a reference trajectory at different time points."""
    return {peptide: compute_JS_divergence_vs_time(trajs[peptide], reference_trajs[peptide], num_steps) for peptide in trajs}


In [38]:
num_steps = 10
JAMUN_js_divs = compute_JS_divergences_vs_time(JAMUN_trajs, Timewarp_trajs, num_steps)
Timewarp_js_divs = compute_JS_divergences_vs_time(Timewarp_trajs, Timewarp_trajs, num_steps)

In [ ]:
for index, peptide in enumerate(JAMUN_js_divs):
    if index == 0:
        Timewarp_label = None
        JAMUN_label = None
    else:
        Timewarp_label = "Uncapped 2AA Reference"
        JAMUN_label = "JAMUN"

    progress = np.arange(num_steps) / num_steps
    plt.plot(progress, Timewarp_js_divs[peptide], color='C0', label=Timewarp_label)
    plt.plot(progress, JAMUN_js_divs[peptide], color='C1', label=JAMUN_label)

plt.title("Jenson-Shannon Divergences")
plt.xlabel("Fraction of Trajectory Progress", fontsize=12)
plt.ylabel("Jenson-Shannon Divergence", fontsize=12)
plt.ticklabel_format(useOffset=False, style='plain')
plt.legend(fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()